In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from datetime import datetime

In [3]:
def load_data():
    print("Loading the athlete performance dataset...")
    df = pd.read_csv("data/athlete_performance_raw.csv")
    print(f"Original dataset shape: {df.shape}")
    return df

In [4]:
# Task 1: Handling Missing Data
def handle_missing_data(df):
    print("\n--- Task 1: Handling Missing Data ---")
    
    # Check for missing values
    missing_values = df.isnull().sum()
    missing_percent = (missing_values / len(df) * 100).round(2)
    
    # Display columns with missing values
    print("Columns with missing values:")
    for col in df.columns:
        if missing_values[col] > 0:
            print(f"  {col}: {missing_values[col]} missing values ({missing_percent[col]}%)")
    
    # Handling strategy for each column with missing values
    
    # 1. Heart_Rate: Impute with median based on similar training sessions (by distance range)
    if missing_values['Heart_Rate'] > 0:
        print("\nImputing missing Heart_Rate values based on similar training distances...")
        
        # Create distance bins
        df['Distance_Bin'] = pd.cut(df['Distance'], bins=5)
        
        # Fill missing heart rates with the median of the same distance bin
        df['Heart_Rate'] = df.groupby('Distance_Bin')['Heart_Rate'].transform(
            lambda x: x.fillna(x.median())
        )
        
        # Remove the temporary bin column
        df = df.drop(columns=['Distance_Bin'])
    
    # 2. Calorie_Intake: Impute with median value for the same food item
    if missing_values['Calorie_Intake'] > 0:
        print("Imputing missing Calorie_Intake values based on the food item...")
        
        # Group by food item to get the median calorie intake
        food_median_calories = df.groupby('Food_Item')['Calorie_Intake'].median()
        
        # For each food item with missing calories, fill with the median for that food
        for food in food_median_calories.index:
            mask = (df['Food_Item'] == food) & (df['Calorie_Intake'].isna())
            df.loc[mask, 'Calorie_Intake'] = food_median_calories[food]
    
    # 3. Hydration_Level: Impute with median based on training effort and distance
    if missing_values['Hydration_Level'] > 0:
        print("Imputing missing Hydration_Level values based on training effort and distance...")
        
        # First, make sure there are no negative hydration levels affecting the median
        hydration_for_median = df[df['Hydration_Level'] > 0]['Hydration_Level']
        median_hydration = hydration_for_median.median()
        
        # Fill missing values with overall median
        df['Hydration_Level'] = df['Hydration_Level'].fillna(median_hydration)
    
    # Check if any missing values remain
    missing_after = df.isnull().sum()
    if missing_after.sum() > 0:
        print("\nRemaining missing values after imputation:")
        for col in df.columns:
            if missing_after[col] > 0:
                print(f"  {col}: {missing_after[col]} missing values")
    else:
        print("\nAll missing values have been handled successfully.")
    
    return df

In [5]:
# Task 2: Removing Duplicate Records
def remove_duplicates(df):
    print("\n--- Task 2: Removing Duplicate Records ---")
    
    # Count initial records
    initial_count = len(df)
    print(f"Initial record count: {initial_count}")
    
    # Check for exact duplicates first
    exact_duplicates = df.duplicated().sum()
    print(f"Exact duplicate records: {exact_duplicates}")
    
    # Define which columns should be considered for identifying training session duplicates
    session_key_columns = ['Session_ID', 'Date', 'Time']
    
    # Find duplicates based on session identifiers
    session_duplicates = df.duplicated(subset=session_key_columns, keep=False)
    
    if session_duplicates.sum() > 0:
        print(f"\nFound {session_duplicates.sum()} records with duplicate session identifiers")
        
        # Display some examples of duplicates
        print("\nSample session duplicates:")
        duplicate_examples = df[session_duplicates].sort_values(by=session_key_columns).head(10)
        print(duplicate_examples[session_key_columns + ['Distance', 'Speed', 'Heart_Rate']].to_string())
        
        # Keep the record with fewer missing values when there are duplicates
        df['missing_count'] = df.isnull().sum(axis=1)
        
        # Sort by session keys and missing count, then remove duplicates
        df = df.sort_values(by=session_key_columns + ['missing_count'])
        df = df.drop_duplicates(subset=session_key_columns, keep='first')
        df = df.drop(columns=['missing_count'])
        
        # Count records after removing duplicates
        final_count = len(df)
        print(f"\nRemoved {initial_count - final_count} duplicate records")
        print(f"Record count after removing duplicates: {final_count}")
    else:
        print("No session duplicates found.")
    
    return df


In [6]:
# Task 3: Standardizing Data Formats
def standardize_formats(df):
    print("\n--- Task 3: Standardizing Data Formats ---")
    
    # 1. Standardize Time format to 24-hour
    print("Standardizing time format to 24-hour...")
    
    # Check current time formats
    time_patterns = df['Time'].str.contains('AM|PM', case=False).sum()
    print(f"Records with 12-hour format (AM/PM): {time_patterns}")
    print(f"Records with 24-hour format: {len(df) - time_patterns}")
    
    # Function to convert 12-hour time to 24-hour format
    def convert_to_24hr(time_str):
        if isinstance(time_str, str) and ('AM' in time_str.upper() or 'PM' in time_str.upper()):
            # Parse 12-hour format
            try:
                if 'AM' in time_str.upper():
                    # Remove AM and handle 12 AM special case
                    time_str = time_str.upper().replace('AM', '').strip()
                    hour, minute = map(int, time_str.split(':'))
                    hour = 0 if hour == 12 else hour
                    return f"{hour:02d}:{minute:02d}"
                else:  # PM
                    # Remove PM and handle 12 PM special case
                    time_str = time_str.upper().replace('PM', '').strip()
                    hour, minute = map(int, time_str.split(':'))
                    hour = hour if hour == 12 else hour + 12
                    return f"{hour:02d}:{minute:02d}"
            except Exception as e:
                print(f"Error converting time: {time_str}, Error: {e}")
                return time_str
        return time_str  # Already in 24-hour format
    
    # Apply the conversion function
    df['Time'] = df['Time'].apply(convert_to_24hr)
    
    # 2. Standardize Distance to kilometers
    print("\nStandardizing distance to kilometers...")
    
    # Check current distance units
    miles_count = (df['Distance_Unit'] == 'miles').sum()
    km_count = (df['Distance_Unit'] == 'km').sum()
    print(f"Records with distance in miles: {miles_count}")
    print(f"Records with distance in kilometers: {km_count}")
    
    # Convert miles to kilometers where needed
    miles_mask = df['Distance_Unit'] == 'miles'
    df.loc[miles_mask, 'Distance'] = df.loc[miles_mask, 'Distance'] * 1.60934
    df.loc[miles_mask, 'Distance_Unit'] = 'km'
    
    # Round distance to 2 decimal places for consistency
    df['Distance'] = df['Distance'].round(2)
    
    # Verify standardization
    print(f"\nAfter standardization, records with distance in km: {(df['Distance_Unit'] == 'km').sum()}")
    
    return df

In [7]:
# Task 4: Identifying and Handling Outliers
def handle_outliers(df):
    print("\n--- Task 4: Identifying and Handling Outliers ---")
    
    # Focus on detecting outliers in the Speed column
    print("Detecting outliers in the Speed column...")
    
    # Plot speed distribution
    plt.figure(figsize=(10, 6))
    sns.boxplot(x=df['Speed'])
    plt.title('Speed Distribution (Before Outlier Handling)')
    plt.savefig('data/speed_distribution_before.png')
    plt.close()
    
    # Calculate IQR for Speed
    Q1 = df['Speed'].quantile(0.25)
    Q3 = df['Speed'].quantile(0.75)
    IQR = Q3 - Q1
    
    # Define bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Identify outliers
    speed_outliers = df[(df['Speed'] < lower_bound) | (df['Speed'] > upper_bound)]
    outlier_count = len(speed_outliers)
    
    print(f"Speed quartiles - Q1: {Q1:.2f}, Median: {df['Speed'].median():.2f}, Q3: {Q3:.2f}")
    print(f"Speed IQR: {IQR:.2f}")
    print(f"Outlier bounds: [{lower_bound:.2f}, {upper_bound:.2f}]")
    print(f"Detected {outlier_count} outliers in Speed column")
    
    if outlier_count > 0:
        # Display outliers
        print("\nSample speed outliers:")
        print(speed_outliers[['Session_ID', 'Date', 'Speed', 'Distance', 'Heart_Rate']].head().to_string())
        
        # Method: Cap outliers at the bounds
        print("\nCapping extreme speed values at the calculated bounds...")
        df['Speed'] = df['Speed'].clip(lower=lower_bound, upper=upper_bound)
        
        # Plot speed distribution after handling outliers
        plt.figure(figsize=(10, 6))
        sns.boxplot(x=df['Speed'])
        plt.title('Speed Distribution (After Outlier Handling)')
        plt.savefig('data/speed_distribution_after.png')
        plt.close()
        
        # Verify outliers were handled
        remaining_outliers = df[(df['Speed'] < lower_bound) | (df['Speed'] > upper_bound)]
        print(f"Remaining outliers after capping: {len(remaining_outliers)}")
    
    return df

In [8]:
# Task 5: Correcting Data Entry Errors
def correct_data_errors(df):
    print("\n--- Task 5: Correcting Data Entry Errors ---")
    
    # 1. Correct incorrect calorie values for food items
    print("Correcting invalid calorie values for food items...")
    
    # Define correct calorie values for common food items
    correct_calories = {
        "Banana": 105,
        "Energy Bar": 250,
        "Oatmeal": 150,
        "Protein Shake": 300,
        "Yogurt": 120,
        "Pasta": 400,
        "Chicken Breast": 165,
        "Salmon": 230,
        "Rice": 210,
        "Sweet Potato": 115
    }
    
    # Identify anomalous calorie values (much higher than expected)
    anomalies = 0
    for food, correct_value in correct_calories.items():
        # Consider values significantly higher than correct as errors
        mask = (df['Food_Item'] == food) & (df['Calorie_Intake'] > correct_value * 5)
        anomaly_count = mask.sum()
        
        if anomaly_count > 0:
            anomalies += anomaly_count
            print(f"  Found {anomaly_count} incorrect calorie values for {food}")
            df.loc[mask, 'Calorie_Intake'] = correct_value
    
    print(f"Total incorrect calorie values corrected: {anomalies}")
    
    # 2. Fix negative hydration levels
    negative_hydration = (df['Hydration_Level'] < 0).sum()
    if negative_hydration > 0:
        print(f"\nFound {negative_hydration} records with negative hydration levels")
        
        # Fix by taking the absolute value
        df.loc[df['Hydration_Level'] < 0, 'Hydration_Level'] = df.loc[df['Hydration_Level'] < 0, 'Hydration_Level'].abs()
        
        print("Corrected negative hydration levels by converting to positive values")
    else:
        print("\nNo negative hydration levels found")
    
    return df


In [9]:
# Task 6: Dropping Irrelevant Columns
def drop_irrelevant_columns(df):
    print("\n--- Task 6: Dropping Irrelevant Columns ---")
    
    # Identify columns that don't contribute to performance analysis
    irrelevant_columns = [
        'Athlete_Social_Media_Followers',
        'Weather_Forecast_Accuracy',
        'Local_Race_Participants',
        'Training_Plan_Version'
    ]
    
    # Check if these columns exist in the dataset
    existing_irrelevant = [col for col in irrelevant_columns if col in df.columns]
    
    if existing_irrelevant:
        print(f"Dropping {len(existing_irrelevant)} irrelevant columns:")
        for col in existing_irrelevant:
            print(f"  - {col}")
        
        # Drop the irrelevant columns
        df = df.drop(columns=existing_irrelevant)
        
        print(f"\nRemaining columns after dropping irrelevant ones: {df.shape[1]}")
    else:
        print("No irrelevant columns found")
    
    return df

In [10]:
# Task 7: Final Dataset Preparation
def prepare_final_dataset(df):
    print("\n--- Task 7: Final Dataset Preparation ---")
    
    # 1. Ensure proper data types
    print("Ensuring proper data types...")
    
    # Convert numeric columns to appropriate types
    numeric_columns = ['Distance', 'Speed', 'Heart_Rate', 'Calories_Burned', 
                       'Temperature', 'Humidity', 'Calorie_Intake', 
                       'Hydration_Level', 'Training_Effort', 'Session_Rating',
                       'Post_Recovery_Score']
    
    for col in numeric_columns:
        if col in df.columns:
            if col in ['Heart_Rate', 'Calories_Burned', 'Calorie_Intake', 
                       'Training_Effort', 'Session_Rating', 'Post_Recovery_Score']:
                df[col] = df[col].astype(int)
            else:
                df[col] = df[col].astype(float)
    
    # 2. Create derived metrics for analysis
    print("\nCreating derived metrics for analysis...")
    
    # Calculate calories burned per kilometer
    df['Calories_Per_Km'] = (df['Calories_Burned'] / df['Distance']).round(1)
    
    # Calculate training efficiency (speed relative to heart rate)
    df['Training_Efficiency'] = (df['Speed'] / df['Heart_Rate'] * 100).round(2)
    
    # Calculate hydration ratio (hydration level per kilometer)
    df['Hydration_Ratio'] = (df['Hydration_Level'] / df['Distance']).round(2)
    
    # 3. Sort dataset by date and time for better analysis
    df = df.sort_values(by=['Date', 'Time'])
    
    # 4. Generate a final dataset summary
    print("\nFinal dataset info:")
    print(df.info())
    
    print("\nFinal dataset descriptive statistics:")
    print(df.describe().round(2))
    
    # 5. Save the cleaned dataset
    df.to_csv('data/athlete_performance_cleaned.csv', index=False)
    print("\nFinal cleaned dataset saved to: data/athlete_performance_cleaned.csv")
    
    return df

In [11]:
# Main function to run all cleaning tasks
def main():
    # Load the data
    df = load_data()
    
    # Apply each cleaning task
    df = handle_missing_data(df)
    df = remove_duplicates(df)
    df = standardize_formats(df)
    df = handle_outliers(df)
    df = correct_data_errors(df)
    df = drop_irrelevant_columns(df)
    df = prepare_final_dataset(df)
    
    print("\nData cleaning complete!")
    
    # Generate a simple summary of changes made
    print("\nSummary of Cleaning Actions:")
    print("1. Handled missing values in Heart_Rate, Calorie_Intake, and Hydration_Level columns")
    print("2. Removed duplicate training sessions")
    print("3. Standardized time format to 24-hour and distance to kilometers")
    print("4. Identified and handled outliers in the Speed column")
    print("5. Corrected erroneous calorie values and negative hydration levels")
    print("6. Removed irrelevant columns not related to performance analysis")
    print("7. Created derived metrics for deeper analysis")

if __name__ == "__main__":
    main() 

Loading the athlete performance dataset...
Original dataset shape: (110, 22)

--- Task 1: Handling Missing Data ---
Columns with missing values:
  Heart_Rate: 15 missing values (13.64%)
  Calorie_Intake: 10 missing values (9.09%)
  Hydration_Level: 13 missing values (11.82%)
  Notes: 17 missing values (15.45%)

Imputing missing Heart_Rate values based on similar training distances...
Imputing missing Calorie_Intake values based on the food item...
Imputing missing Hydration_Level values based on training effort and distance...

Remaining missing values after imputation:
  Notes: 17 missing values

--- Task 2: Removing Duplicate Records ---
Initial record count: 110
Exact duplicate records: 0

Found 20 records with duplicate session identifiers

Sample session duplicates:
     Session_ID        Date     Time  Distance  Speed  Heart_Rate
17           18  2023-01-18    19:15     11.92   3.07       141.0
105          18  2023-01-18    19:15     11.92   3.07       141.0
19           20  202

/var/folders/rw/g4p48pvn7w946c84lc3b8sdc0000gn/T/ipykernel_66011/3899851911.py:25: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['Heart_Rate'] = df.groupby('Distance_Bin')['Heart_Rate'].transform(
